In [1]:
import pandas as pd

In [2]:
def get_avg_use_per_bomb_in_minutes_corrected() -> pd.DataFrame:
    df = pd.read_parquet("../data/silver/water_consumption_silver.parquet")
    
    # Ensure timestamp is in datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Create date column
    df['date'] = df['timestamp'].dt.date
    
    # Calculate peak hours
    peak_hours = (df["hour"] >= 18) & (df["hour"] <= 21)
    df["is_peak_hour"] = peak_hours
    
    # Calculate duration of pumps
    df['pump_1_duration'] = df['pump_1'] * 3600
    df['pump_2_duration'] = df['pump_2'] * 3600
    
    # Sum water bombs usage time per day and hour
    daily_peak_usage = df[df['is_peak_hour']].groupby('date').agg({'pump_1_duration': 'sum', 'pump_2_duration': 'sum'})
    daily_off_peak_usage = df[~df['is_peak_hour']].groupby('date').agg({'pump_1_duration': 'sum', 'pump_2_duration': 'sum'})

    # Calculate water bombs average usage time per day in minutes
    gmb_1_peak_avg = daily_peak_usage['pump_1_duration'].mean() / 60  # convert seconds to minutes
    gmb_1_off_peak_avg = daily_off_peak_usage['pump_1_duration'].mean() / 60  # convert seconds to minutes

    gmb_2_peak_avg = daily_peak_usage['pump_2_duration'].mean() / 60  # convert seconds to minutes
    gmb_2_off_peak_avg = daily_off_peak_usage['pump_2_duration'].mean() / 60  # convert seconds to minutes
    
    def convert_to_hours_and_minutes(minutes):
        if pd.isna(minutes):
            return "0 hours and 0 minutes"
        total_minutes = int(minutes)
        hours = total_minutes // 60
        minutes = total_minutes % 60
        return f"{hours} hours and {minutes} minutes"
    
    data = {
        'pump': ['pump_1', 'pump_2'],
        'average_time_used_peak_hours': [
            convert_to_hours_and_minutes(gmb_1_peak_avg), 
            convert_to_hours_and_minutes(gmb_2_peak_avg)
        ],
        'average_time_used_offpeak_hours': [
            convert_to_hours_and_minutes(gmb_1_off_peak_avg), 
            convert_to_hours_and_minutes(gmb_2_off_peak_avg)
        ]
    }
    
    result_df = pd.DataFrame(data)
    result_df.to_parquet("../data/gold/question_3_answer.parquet")


In [3]:
df = pd.read_parquet("../data/silver/water_consumption_silver.parquet")
df.head()

,timestamp,hour,day_of_week,week_of_year,year,input_flow_rate,reservoir_level_percentage,pressure,output_flow_rate,air_temp_c,total_precip_mm,relative_humidity_percentage,pump_1,pump_2
0,2023-03-17 12:00:00,12,4,11,2023,66.13,35.86,38.08,49.69,27.9,0.0,70.0,0.0,0.48
1,2023-03-17 13:00:00,13,4,11,2023,65.59,39.00,38.18,50.70,28.9,0.0,59.0,0.0,1.00
2,2023-03-17 14:00:00,14,4,11,2023,65.60,44.40,38.46,50.21,29.9,0.0,54.0,0.0,0.98
3,2023-03-17 15:00:00,15,4,11,2023,70.67,50.56,38.56,70.66,30.4,0.0,53.0,0.0,1.00
4,2023-03-17 16:00:00,16,4,11,2023,6.33,53.80,32.05,48.68,31.2,0.0,57.0,0.0,0.18


In [4]:
get_avg_use_per_bomb_in_minutes_corrected()

In [5]:
df = pd.read_parquet("../data/gold/question_3_answer.parquet")
df.head()

,pump,average_time_used_peak_hours,average_time_used_offpeak_hours
0,pump_1,1 hours and 1 minutes,5 hours and 6 minutes
1,pump_2,0 hours and 54 minutes,4 hours and 22 minutes
